## (i) Importing the necessary packages

In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten,Dropout
from keras.layers import Conv2D,BatchNormalization,MaxPooling2D,Reshape
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [27]:
import sys
runningOnColab = 'google.colab' in sys.modules
if runningOnColab:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd '/content/drive/Shareddrives/ECE247/Project'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/ECE247/Project


## (ii) Preprocessing the dataset and preparing the training, validation, and test datasets

In [28]:
## Loading and visualizing the data
## Loading the dataset
X_test_load = np.load("X_test.npy")
y_test_load = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid_load = np.load("X_train_valid.npy")
y_train_valid_load = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

person_train_valid = np.squeeze(person_train_valid)
person_test = np.squeeze(person_test)

y_train_valid_load -= 769
y_test_load -= 769

## (iii)(CNN) Defining the architecture of a basic CNN model

In [29]:
def create_model(t):
    # Building the CNN model using sequential class
    basic_cnn_model = Sequential()

    # Conv. block 1
    basic_cnn_model.add(Conv2D(filters=25, kernel_size=(10,1), padding='same', activation='elu', input_shape=(t,1,22)))
    basic_cnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same')) # Read the keras documentation
    basic_cnn_model.add(BatchNormalization())
    basic_cnn_model.add(Dropout(0.5))

    # Conv. block 2
    basic_cnn_model.add(Conv2D(filters=50, kernel_size=(10,1), padding='same', activation='elu'))
    basic_cnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
    basic_cnn_model.add(BatchNormalization())
    basic_cnn_model.add(Dropout(0.5))

    # Conv. block 3
    basic_cnn_model.add(Conv2D(filters=100, kernel_size=(10,1), padding='same', activation='elu'))
    basic_cnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
    basic_cnn_model.add(BatchNormalization())
    basic_cnn_model.add(Dropout(0.5))

    # Conv. block 4
    basic_cnn_model.add(Conv2D(filters=200, kernel_size=(10,1), padding='same', activation='elu'))
    basic_cnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
    basic_cnn_model.add(BatchNormalization())
    basic_cnn_model.add(Dropout(0.5))

    # Output layer with Softmax activation
    basic_cnn_model.add(Flatten()) # Flattens the input
    basic_cnn_model.add(Dense(4, activation='softmax')) # Output FC layer with softmax activation

    return basic_cnn_model


## (iv)(CNN) Defining the hyperparameters of the basic CNN model

In [30]:
# Model parameters
learning_rate = 1e-3
epochs = 50
cnn_optimizer = tf.keras.optimizers.Adam(lr=learning_rate)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


![EEG_prep.png](attachment:EEG_prep.png)

In [31]:
def data_prep(X, y, person, t, sub_sample, average, noise):
    total_X = None
    total_y = None
    total_person = None

    # Trimming the data (sample,22,1000) -> (sample,22,500)
    X = X[:,:,0:t]
    
    # Maxpooling the data (sample,22,1000) -> (sample,22,500/sub_sample)
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, sub_sample), axis=3)
    
    total_X = X_max
    total_y = y
    total_person = person
    
    # Averaging + noise 
    X_average = np.mean(X.reshape(X.shape[0], X.shape[1], -1, average),axis=3)
    X_average = X_average + np.random.normal(0.0, 0.5, X_average.shape)
    
    total_X = np.vstack((total_X, X_average))
    total_y = np.hstack((total_y, y))
    total_person = np.hstack((total_person, person))
    
    # Subsampling
    
    for i in range(sub_sample):
        
        X_subsample = X[:, :, i::sub_sample] + \
                            (np.random.normal(0.0, 0.5, X[:, :,i::sub_sample].shape) if noise else 0.0)
            
        total_X = np.vstack((total_X, X_subsample))
        total_y = np.hstack((total_y, y))
        total_person = np.hstack((total_person, person))
        
    return total_X, total_y, total_person 


In [32]:

## Preprocessing the dataset
times = [400, 500, 600, 700]

for t in times:
    print('Time: ', t)
    X_train_valid_prep, y_train_valid_prep, person_train_valid_prep = data_prep(X_train_valid_load, y_train_valid_load, person_train_valid, t, 2, 2, True)
    X_test_prep, y_test_prep, person_test_prep = data_prep(X_test_load, y_test_load, person_test, t, 2, 2, True)

    ## Random splitting and reshaping the data
    # First generating the training and validation indices using random splitting
    ind_valid = np.random.choice(8460, 1500, replace=False)
    ind_train = np.array(list(set(range(8460)).difference(set(ind_valid))))

    # Creating the training and validation sets using the generated indices
    (x_train, x_valid) = X_train_valid_prep[ind_train], X_train_valid_prep[ind_valid] 
    (y_train, y_valid) = y_train_valid_prep[ind_train], y_train_valid_prep[ind_valid]
    (person_train, person_valid) = person_train_valid_prep[ind_train], person_train_valid_prep[ind_valid]


    # Converting the labels to categorical variables for multiclass classification
    y_train = to_categorical(y_train, 4)
    y_valid = to_categorical(y_valid, 4)
    y_test = to_categorical(y_test_prep, 4)

    person_test = person_test_prep

    # Adding width of the segment to be 1
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1], x_train.shape[2], 1)
    x_test = X_test_prep.reshape(X_test_prep.shape[0], X_test_prep.shape[1], X_test_prep.shape[2], 1)

    # Reshaping the training and validation dataset
    x_train = np.swapaxes(x_train, 1,3)
    x_train = np.swapaxes(x_train, 1,2)
    x_valid = np.swapaxes(x_valid, 1,3)
    x_valid = np.swapaxes(x_valid, 1,2)
    x_test = np.swapaxes(x_test, 1,3)
    x_test = np.swapaxes(x_test, 1,2)

    basic_cnn_model = create_model(t // 2)
    # Compiling the model
    basic_cnn_model.compile(loss='categorical_crossentropy',
                    optimizer=cnn_optimizer,
                    metrics=['accuracy'])

    # Training and validating the model
    basic_cnn_model_results = basic_cnn_model.fit(x_train,
                y_train,
                batch_size=64,
                epochs=epochs,
                validation_data=(x_valid, y_valid), verbose=False)

    cnn_score = basic_cnn_model.evaluate(x_test, y_test, verbose=0)
    print('Test accuracy of the basic CNN model:',cnn_score[1]) 


    



Time:  400
Test accuracy of the basic CNN model: 0.709932267665863
Time:  500
Test accuracy of the basic CNN model: 0.7093679308891296
Time:  600
Test accuracy of the basic CNN model: 0.6867945790290833
Time:  700
Test accuracy of the basic CNN model: 0.6828442215919495
